1. 문서 내용 읽기
2. 문서 분할(쪼개기)
    - 문서를 분할하지 않으면,
        - 토큰 수 초과로 답변을 생성하지 못할 수 있음
        -문서 길이(input)가 길어서 답변 생성에 시간 소요됨 -> 답변 늦어짐
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있으면, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [13]:
%pip install -Uq langchain langchain-community


Note: you may need to restart the kernel to use updated packages.


In [29]:
## 패키지/라이브러리 설치
# %pip install -qU docx2txt langchain_community langchain-text-splitters langchain-chroma

##

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI

# 1. 문서 내용 읽고 분할 ########################################
loader = Docx2txtLoader('law_1.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

# 2. 임베딩 -> 벡터 데이터베이스에 저장 ######################################

## 2-1 환경변수 읽어오기
load_dotenv()

## 2-2. 임베딩 모델 지정 
embedding = OpenAIEmbeddings(model='text-embedding-3-large')


## 2-3. 벡터 데이터베이스에 저장 
## [방법 1] in memory
database = Chroma.from_documents(
    documents=document_list,
    embedding=embedding,
)



## 3. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색 ############################
## 3-1. 사용자 질문
query = '전세사기피해자 금융지원에 대해 알려다오'

## 3-2. 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query)

## 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 #############################
## 4-1 프롬프트 작성 
prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
-[context]를 참고하여 사용자의 질문에 답변해주세요.

[content]
{retrieved_docs}

Question: {query}

'''
## 4-2. 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(
    retrieved_docs=retrieved_docs,
    query=query
)


from langchain_openai import ChatOpenAI

## 4-3. LLM 모델 생성(chatOpenAI 인스턴스 생성)
llm = ChatOpenAI(model='gpt-4o')
## 4-4. LLM 모델에 질문과 검색된 문서를 보냄
ai_message = llm.invoke(formatted_prompt)
ai_message

AIMessage(content='전세사기피해자에 대한 금융지원은 "전세사기피해자 지원 및 주거안정에 관한 특별법"에 따라 다양한 형태로 이루어질 수 있습니다. 다음은 해당 법률에서 제공하는 금융지원에 대한 주요 내용입니다:\n\n1. **융자 지원**:\n   - 국가 및 지방자치단체는 전세사기피해자 및 긴박한 주거안정이 필요한 임차인에게 필요한 자금을 융자하거나 기타 필요한 지원을 제공할 수 있습니다.\n   - 특히 주택임대차보호법에 따른 우선변제를 받지 못해 시급한 지원이 필요한 전세사기피해자에게는 주택도시기금에서 주택의 임대차에 필요한 자금을 융자할 수 있습니다.\n\n2. **신용정보 관련 지원**:\n   - 전세사기피해자의 보증금이 변제되지 않은 경우, 해당 전세 관련 대출에 대한 채무 불이행 및 대위변제를 등록 유예하거나 삭제할 수 있습니다. 이 조치는 신용정보집중기관을 통해 정한 기준에 따라 이루어집니다.\n\n3. **주택저당채권대상 확대**:\n   - 전세사기피해자 및 관련 임차인에 대해 한국주택금융공사가 제공하는 주택저당채권대상에 주거 목적으로 사용되는 오피스텔이 포함될 수 있습니다.\n\n이러한 금융지원 방안은 전세사기로 인한 경제적 어려움을 해결하고 주거 안정을 도모하기 위한 국가와 공공기관의 지원 정책입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 351, 'prompt_tokens': 4166, 'total_tokens': 4517, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'm